# Geological Interpretor Development

This is a notebook for testing and developping some of the basic code in this package.

## Testing ontology manipulation

The knowledge manipulated in this package is formalised in an ontology,<br>
which is store in a *.owl* file.

It is named **MOGI** for **M**inimal **O**ntology for **G**eological **I**nterpretation

To manipulated this ontology, we use the package **owlready2** available from here: https://owlready2.readthedocs.io

In [ ]:
import owlready2 as owl

In [ ]:
owl.onto_path.append("../ontologies/")
onto = owl.get_ontology("mogi.owl").load()
onto

Ontology provides access to its components, e.g.:
* classes
* properties
* individuals
* rules

In [ ]:
print(list(onto.classes()))
print(list(onto.properties()))
print(list(onto.individuals()))
print(list(onto.rules()))

More specific elements can be searched through simple queries:

In [ ]:
onto.search(iri = "*Surface")

### Reasoner

Ontologies are even more powerful thansk to their capabilities to use reasoning for infering types, properties, and relationships that were not explicitly stated.
This is usefull for obtaining results implied by the already stated information.

This is achieved by running a *reasoner* on the ontology as follows.

In [ ]:
owl.sync_reasoner(infer_property_values=True)

In [ ]:
Dtest = onto.Data("Dtest",dip_dir=[270],dip=[45],x=[15],y=[5],z=[25])

In [ ]:
for prop in onto.Dtest.get_properties():
    for value in prop[onto.Dtest]:
        print(prop,":",value)

In [ ]:
onto.dip_dir[onto.Dtest]